In [1]:
import numpy as np
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import pickle
from pathlib import Path
import os
from tqdm import tqdm
import sys

In the end augmentations did not end up helping momask, and were not generated for the other datasets. In the new iteration of momask where we abandon the 256 hidden layer in favor of just a single fully connected layer we will use just the original version at 30fps. We will keep the old code that supports adding precomputed augmentations but will just use a dataset which has no augmentations.

## CONFIG

In [ ]:

HUMANML3D_COLLECTED_OUTPUTS = {
    'BMCLab': './datasets/HumanML3D/BMCLab//HumanML3D_collected.npz',
    '3DGait': './datasets/HumanML3D/3DGait/HumanML3D_collected.npz',
    'PD-GaM': './datasets/HumanML3D/PD-GaM/HumanML3D_collected.npz',
    'T-SDU-PD': './datasets/HumanML3D/T-SDU-PD/HumanML3D_collected.npz',
}

## Calculating Mean and Std for humanml3d representations

In [42]:
# root_rot_velocity (B, seq_len, 1)
# root_linear_velocity (B, seq_len, 2)
# root_y (B, seq_len, 1)
# ric_data (B, seq_len, (joint_num - 1)*3)
# rot_data (B, seq_len, (joint_num - 1)*6)
# local_velocity (B, seq_len, joint_num*3)
# foot contact (B, seq_len, 4)
OPT_FEAT_BIAS = 5 # In Dataset object they divide some std enteries further by 5 which comes from the opt file https://github.com/EricGuo5513/momask-codes/blob/main/data/t2m_dataset.py#L41
def mean_variance(npz_dirs, save_dir, joints_num, save_dir2):
    data_list = []
    for npz_dir in npz_dirs:
        humanml3d_dataset = np.load(npz_dir, allow_pickle=True)
        for name in humanml3d_dataset:
            data = humanml3d_dataset[name]
            if data.ndim == 2:
                data = np.expand_dims(data, axis=0)
            data = data[0] # Get rid of additional first axis that is legacy from when it used to denote different augmentations
            if np.isnan(data).any():
                print(f"{npz_dir}, {name}")
                continue
            data_list.append(data)

    data = np.concatenate(data_list, axis=0)
    print(f'concatinated data shape: {data.shape}')
    Mean = data.mean(axis=0)
    Std = data.std(axis=0)
    Std[0:1] = Std[0:1].mean() / OPT_FEAT_BIAS
    Std[1:3] = Std[1:3].mean() / OPT_FEAT_BIAS
    Std[3:4] = Std[3:4].mean() / OPT_FEAT_BIAS
    Std[4: 4+(joints_num - 1) * 3] = Std[4: 4+(joints_num - 1) * 3].mean() / 1.0
    Std[4+(joints_num - 1) * 3: 4+(joints_num - 1) * 9] = Std[4+(joints_num - 1) * 3: 4+(joints_num - 1) * 9].mean() / 1.0
    Std[4+(joints_num - 1) * 9: 4+(joints_num - 1) * 9 + joints_num*3] = Std[4+(joints_num - 1) * 9: 4+(joints_num - 1) * 9 + joints_num*3].mean() / 1.0
    Std[4 + (joints_num - 1) * 9 + joints_num * 3: ] = Std[4 + (joints_num - 1) * 9 + joints_num * 3: ].mean() / OPT_FEAT_BIAS

    assert 8 + (joints_num - 1) * 9 + joints_num * 3 == Std.shape[-1]

    np.save(save_dir / 'Mean.npy', Mean)
    np.save(save_dir / 'Std_FEAT_BIAS_included.npy', Std)
    
    np.save(save_dir2 / 'Mean.npy', Mean)
    np.save(save_dir2 / 'Std_FEAT_BIAS_included.npy', Std)

    return Mean, Std

### For each individual dataset

In [43]:
for dataset in HUMANML3D_COLLECTED_OUTPUTS:
    save_dir = Path(HUMANML3D_COLLECTED_OUTPUTS[dataset]).parent
    print(str(save_dir).split('datasets'))
    save_dir2 = Path(str(save_dir).split('datasets')[0] + 'datasets/HumanML3D_norm_data/' + dataset)
    npz_dirs = [Path(HUMANML3D_COLLECTED_OUTPUTS[dataset])]
    print(save_dir)
    print(save_dir2)
    print(npz_dirs)
    print(f'Processing {dataset}:')
    Mean, Std = mean_variance(npz_dirs, save_dir, 22, save_dir2)
    print(f"For {dataset} Mean and Std_FEAT_BIAS_included are:")
    print("Mean:")
    print(Mean)
    print()
    print("Std feat_bias included:")
    print(Std)
    print()
    print()
    

['/mnt/vida/Work/AMBIENT/Code/datasetpaper/ArchGait/Ndrive/', '/BMClab/HumanML3D']
/mnt/vida/Work/AMBIENT/Code/datasetpaper/ArchGait/Ndrive/datasets/BMClab/HumanML3D
/mnt/vida/Work/AMBIENT/Code/datasetpaper/ArchGait/Ndrive/datasets/HumanML3D_norm_data/BMCLABS
[PosixPath('/mnt/vida/Work/AMBIENT/Code/datasetpaper/ArchGait/Ndrive/datasets/BMClab/HumanML3D/HumanML3D_collected.npz')]
Processing BMCLABS:
concatinated data shape: (834893, 263)
For BMCLABS Mean and Std_FEAT_BIAS_included are:
Mean:
[ 2.92458662e-06 -3.43726751e-05  2.88738441e-02  9.56831992e-01
  6.05797544e-02  8.76007617e-01 -3.79053969e-03 -6.45398870e-02
  8.71284604e-01 -4.14961250e-03  1.17585751e-05  1.08360028e+00
 -3.35337035e-02  1.14507429e-01  5.05914152e-01  8.56147408e-02
 -1.18275449e-01  5.03760934e-01  8.44070464e-02 -1.08505969e-06
  1.22702992e+00 -6.48689922e-04  7.86858797e-02  1.15336224e-01
 -4.89831753e-02 -8.26887339e-02  1.19248770e-01 -4.80300821e-02
 -3.15822217e-05  1.26958668e+00  3.18108797e-02 

### For Leave-One-Dataset-Out merged datasets

In [45]:
for lodo_dataset in HUMANML3D_COLLECTED_OUTPUTS:
    all_other_datasets = [d for d in HUMANML3D_COLLECTED_OUTPUTS if d != lodo_dataset]
    npz_dirs = [Path(HUMANML3D_COLLECTED_OUTPUTS[dataset]) for dataset in all_other_datasets]
    save_dir = Path(HUMANML3D_COLLECTED_OUTPUTS[dataset]).parents[2] / 'HumanML3D' / f"LODO_{lodo_dataset}"
    save_dir2 = Path(str(save_dir).split('datasets')[0] + 'datasets/HumanML3D_norm_data/' + f"LODO_{lodo_dataset}")
    # print(save_dir)
    # print(save_dir2)
    os.makedirs(save_dir, exist_ok=True)
    Mean, Std = mean_variance(npz_dirs, save_dir2, 22, save_dir2)
    print(f"For merged datasets {all_other_datasets} (LODO {lodo_dataset}) Mean and Std_FEAT_BIAS_included are:")
    print("Mean:")
    print(Mean)
    print()
    print("Std feat_bias included:")
    print(Std)
    print()
    print()

concatinated data shape: (1142490, 263)
For merged datasets ['3DGAIT', 'KIEL', 'PDGAM', 'TRI_PD'] (LODO BMCLABS) Mean and Std_FEAT_BIAS_included are:
Mean:
[-9.72943308e-05 -2.12791565e-04  3.19021717e-02  1.00158775e+00
  6.04423992e-02  9.17860448e-01 -4.93982295e-03 -6.40204772e-02
  9.12631750e-01 -4.16533416e-03 -1.28316286e-03  1.13222003e+00
 -3.13475840e-02  9.97444689e-02  5.45241237e-01  5.16381711e-02
 -1.02210283e-01  5.42330921e-01  5.03412075e-02  9.73177492e-04
  1.27002227e+00 -6.39460122e-05  7.15349689e-02  1.55125976e-01
 -7.49924704e-02 -7.06093982e-02  1.59409180e-01 -7.68990517e-02
  1.21437490e-03  1.30586231e+00  3.55614386e-02  9.66719985e-02
  7.85706788e-02  3.34150828e-02 -1.02322944e-01  8.24324116e-02
  3.68751809e-02  1.40593285e-04  1.52715063e+00  5.33701405e-02
  7.38582239e-02  1.42210877e+00  3.45512182e-02 -7.53743127e-02
  1.42549300e+00  3.21292356e-02  1.34919677e-03  1.57760036e+00
  1.35977730e-01  2.02869430e-01  1.41981494e+00  3.28659341e-02

## Sainty Check PD4T

In [20]:
vidas_mean = np.load('/Volumes/AIRR/AMBIENT/Vida/Unified_Motion_DB/PD4T/HumanML3Drep_30fps_score3adjusted/Mean.npy')
vidas_std = np.load('/Volumes/AIRR/AMBIENT/Vida/Unified_Motion_DB/PD4T/HumanML3Drep_30fps_score3adjusted/Std.npy')

In [21]:
collected = dict()
for path in Path(HUMANML3D_DATASET_PATHS['PD4T']).rglob('*.npy'):
    humanml3d_seq = np.load(path)
    name = path.stem
    collected[name] = humanml3d_seq[np.newaxis, ...]
np.savez('/Users/klabs/Downloads/PD4T_all_humanml3d_collected.npz', **collected)

In [22]:
ivans_mean, ivans_std = mean_variance([Path('/Users/klabs/Downloads/PD4T_all_humanml3d_collected.npz')], Path('/Users/klabs/Downloads'), 22)

(279539, 263)


In [23]:
np.allclose(vidas_mean, ivans_mean)

True

When calculated on ALL PD4T sequences (even those that get discarded because they are shorther than 30 frames my and vidas calculated mean vector matches) so the mean_variance method must be okay